In [1]:
# %load ../env.py
%reload_ext autoreload
%autoreload 2
%pylab inline
%matplotlib inline

import matplotlib.text as text
import pandas as pd
import numpy as np
import pylab as pl
import scipy as sp
import sys
import rpy2 
import os 

#Set environment variables


# Set up the local source files
TOP = '/'.join(os.getcwd().split('/')[:-2])+'/'
#TOP = "/share/home/ishah/ipynb/pb/HTTR_WF/"

LIB = TOP+'lib'
if not LIB in sys.path: 
    sys.path.insert(0,LIB)

os.environ['PYTHONPATH']=LIB


DAT_DIR = TOP + '/data/'
FIG_DIR = TOP + '/figs/'

if not os.path.exists(DAT_DIR): os.mkdir(DAT_DIR)
if not os.path.exists(FIG_DIR): os.mkdir(FIG_DIR)


from db.mongo import *


Populating the interactive namespace from numpy and matplotlib


# Summary

Create the new version of the GenRA database. The last version is genra_v3 so this will be genra_dev_v4. The keys steps are:

* Chemical data: load chemical structure data from DSSTox and create chemical fingerprints
* Bioactivity data: load bioactivity data from different sources (ToxCast, Tox21, other?) and create bioactivity fingerprints
* Toxicity data: load toxicity data from toxrefdb and create toxicity fingerprints
* Physprop data: load physprob data from DSSTox

# Chemical Data

## Loading from DSSTox
Todo: describe how the chemical information is loaded into the database and kept updated. 

Right now just copy over collection that is already setup

`mongodump -j 10 -d ghelman -c compounds -o dump`

`mongodump -j 10 -d ghelman -c logs -o dump`

`mongorestore -j 10 -d genra_dev_v4 -c compounds dump/compounds.bson`

`mongorestore -j 10 -d genra_dev_v4 -c logs dump/logs.bson`


In [3]:
DB = openMongo(db='genra_dev_v4')
GH = openMongo(db='ghelman')
GH.collection_names()

[u'physprop',
 u'test_compounds',
 u'logs',
 u'toxval',
 u'test_physprop',
 u'fingerprints',
 u'meta',
 u'invitrodb',
 u'test_fp',
 u'compounds']

In [8]:
DB.compounds.find_one({'name':'Bisphenol A'},dict(synonyms=0))

{u'_id': ObjectId('58fe610bf0e291b4c06a1aec'),
 u'casrn': u'80-05-7',
 u'chemspider_id': 6371,
 u'created_at': datetime.datetime(2017, 4, 24, 20, 32, 19),
 u'dsstox_cid': u'DTXCID30182',
 u'dsstox_sid': u'DTXSID7020182',
 u'gsid': 20182,
 u'inchi_key': u'IISBACLAFKSPIT-UHFFFAOYSA-N',
 u'iupac': u"4,4'-(Propane-2,2-diyl)diphenol",
 u'mol_weight': 228.29100036621094,
 u'name': u'Bisphenol A',
 u'pubchem_cid': 6623,
 u'smiles': u'CC(C)(C1=CC=C(O)C=C1)C1=CC=C(O)C=C1',
 u'updated_at': datetime.datetime(2017, 8, 30, 19, 10, 40)}

In [10]:
DB.logs.find_one(dict(dsstox_cid='DTXCID30182'))

{u'_id': ObjectId('58fe610bf0e291b4c06a1aee'),
 u'dsstox_cid': u'DTXCID30182',
 u'logs': [u'Added to database',
  {u'collection': u'compounds',
   u'compound': u'DTXCID30182',
   u'field': u'predicted_props',
   u'prop_name': u'ACD_BP',
   u'time': datetime.datetime(2017, 7, 7, 18, 24, 11),
   u'type': u'add',
   u'value': 400.837},
  {u'collection': u'compounds',
   u'compound': u'DTXCID30182',
   u'field': u'predicted_props',
   u'prop_name': u'ACD_FP',
   u'time': datetime.datetime(2017, 7, 7, 18, 24, 11),
   u'type': u'add',
   u'value': 192.423},
  {u'collection': u'compounds',
   u'compound': u'DTXCID30182',
   u'field': u'predicted_props',
   u'prop_name': u'ACD_LogP_v_LogP_Classic',
   u'time': datetime.datetime(2017, 7, 7, 18, 24, 11),
   u'type': u'add',
   u'value': 3.431},
  {u'collection': u'compounds',
   u'compound': u'DTXCID30182',
   u'field': u'predicted_props',
   u'prop_name': u'ACD_Prop_Parachor',
   u'time': datetime.datetime(2017, 7, 26, 18, 24, 49),
   u'type': 

In [ ]:
for C in DB.compounds.find({},dict(_id=0,synonyms=0),no_cursor_timeout=True):
    L = DB.logs.update_one(dict(dsstox_cid=C['dsstox_cid']),{'$set':{'dsstox_sid':C['dsstox_sid']}})
